In [ ]:
!pip install setfit datasets sentence-transformers pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers.losses import CosineSimilarityLoss

# 1. Download the real-world dataset
print("Downloading real customer support data...")
# This dataset contains 26k+ generic support queries
dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")
df = pd.DataFrame(dataset['train'])

print(f"Data Loaded. Rows: {len(df)}")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Bitext_Sample_Customer_Support_Training_(…):   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Data Loaded. Rows: 26872


In [ ]:
import pandas as pd
from datasets import Dataset

label_mapping = {
    "payment_issue": "Billing/Invoice",
    "check_invoice": "Billing/Invoice",
    "get_invoice": "Billing/Invoice",
    "check_payment_methods": "Billing/Invoice",

    "create_account": "Access Request",
    "delete_account": "Access Request",
    "edit_account": "Access Request",
    "recover_password": "Access Request",
    "registration_problems": "Access Request",

    "complaint": "Complaint/Escalation",
    "review": "Complaint/Escalation",

    "contact_customer_service": "General Inquiry",
    "check_refund_policy": "General Inquiry",
    "delivery_options": "General Inquiry",
    "delivery_period": "General Inquiry",

    "newsletter_subscription": "Technical Support",
    "contact_human_agent": "Technical Support",
}

df['label'] = df['intent'].map(label_mapping)
real_data = df.dropna(subset=['label'])
real_data = real_data[['instruction', 'label']].rename(columns={'instruction': 'text'})

real_data = real_data.groupby('label').head(80)

# 2. INJECT SYNTHETIC DATA (English + PROPER HINDI)
# Using Devanagari script allows the model to leverage its full multilingual training.
synthetic_data = [
    # --- Billing/Invoice (Hindi) ---
    {"text": "मुझे पिछले महीने का बिल भेजें।", "label": "Billing/Invoice"},
    {"text": "मेरे पैसे कट गए पर रिचार्ज नहीं हुआ।", "label": "Billing/Invoice"},
    {"text": "इनवॉइस डाउनलोड नहीं हो रहा है।", "label": "Billing/Invoice"},
    {"text": "आपने गलत राशि चार्ज की है।", "label": "Billing/Invoice"},

    # --- Technical Support (Hindi + English) ---
    {"text": "ऐप बार-बार क्रैश हो रहा है।", "label": "Technical Support"},
    {"text": "लॉगिन पेज नहीं खुल रहा है, एरर 500 आ रहा है।", "label": "Technical Support"},
    {"text": "App crashes when I click export. Error 500", "label": "Technical Support"}, # English specific
    {"text": "API endpoint is timing out.", "label": "Technical Support"}, # English specific

    # --- Access Request (Hindi) ---
    {"text": "मेरा पासवर्ड रीसेट कर दें।", "label": "Access Request"},
    {"text": "मुझे डैशबोर्ड के लिए एडमिन एक्सेस चाहिए।", "label": "Access Request"},
    {"text": "अकाउंट लॉक्ड (locked) बता रहा है।", "label": "Access Request"},

    # --- Feature Request (Hindi + English) ---
    {"text": "क्या आप डार्क मोड जोड़ सकते हैं?", "label": "Feature Request"},
    {"text": "एक्सेल एक्सपोर्ट का फीचर कब आएगा?", "label": "Feature Request"},
    {"text": "Can you add an option to export to Excel?", "label": "Feature Request"}, # English

    # --- Hardware/Infrastructure (Hindi + English) ---
    {"text": "सर्वर डाउन है, इसे जल्दी ठीक करें।", "label": "Hardware/Infrastructure"},
    {"text": "सिस्टम बहुत गर्म हो रहा है, सीपीयू (CPU) यूसेज ज्यादा है।", "label": "Hardware/Infrastructure"},
    {"text": "Our server is down, we need immediate help.", "label": "Hardware/Infrastructure"}, # English
    {"text": "Latency is high on the US-East server.", "label": "Hardware/Infrastructure"}, # English

    # --- Complaint/Escalation (Hindi) ---
    {"text": "सर्विस बहुत बेकार है, मुझे रिफंड चाहिए।", "label": "Complaint/Escalation"},
    {"text": "मैं तीन दिन से इंतजार कर रहा हूं, कोई जवाब नहीं मिला।", "label": "Complaint/Escalation"},
    {"text": "यह स्वीकार्य नहीं है, मैनेजर से बात कराएं।", "label": "Complaint/Escalation"},

    # --- How-To/Documentation (Hindi + English) ---
    {"text": "एपीआई (API) की कैसे जनरेट करें?", "label": "How-To/Documentation"},
    {"text": "इसे कॉन्फ़िगर कैसे करते हैं?", "label": "How-To/Documentation"},
    {"text": "How do I configure the API key?", "label": "How-To/Documentation"}, # English

    # --- Data Request (Hindi + English) ---
    {"text": "मेरा पूरा डेटा एक्सपोर्ट करके भेजें।", "label": "Data Request"},
    {"text": "मुझे CSV में पुरानी सेल्स हिस्ट्री की फाइल चाहिए।", "label": "Data Request"},
    {"text": "Please export the last 6 months of user logs.", "label": "Data Request"}, # English

    # --- General Inquiry (Hindi) ---
    {"text": "ऑफिस कब खुलता है?", "label": "General Inquiry"},
    {"text": "सपोर्ट का समय क्या है?", "label": "General Inquiry"}
]

synthetic_df = pd.DataFrame(synthetic_data)

# Combine Real (English) + Synthetic (Hindi/English)
final_train_df = pd.concat([real_data, synthetic_df]).sample(frac=1, random_state=42)
train_dataset = Dataset.from_pandas(final_train_df)

print(f"Final Training Set Size: {len(train_dataset)} examples (English + Proper Hindi)")
print(f"Categories Covered: {len(final_train_df['label'].unique())}")

Final Training Set Size: 429 examples (English + Proper Hindi)
Categories Covered: 9


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:

# 1. Load the Multilingual Model
model_id = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model = SetFitModel.from_pretrained(model_id)

# 2. Configure the Trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=20,
    num_epochs=1
)

# 3. Start Training
print("Starting Multilingual Training...")
trainer.train()
print("Training Complete!")

# 4. Save the model so you can use it in the testing cell
model.save_pretrained("my_multilingual_model")
print("Model saved to 'my_multilingual_model'")

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Step,Training Loss
1,0.072900
50,0.103000
100,0.027300
150,0.005700
200,0.002800
250,0.002300
300,0.001200
350,0.000900
400,0.000500
450,0.000800


/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py:57: DeprecationWarning: The argument 'device' of Tensor.pin_memory() is deprecated. Please do not pass this argument. (Triggered internally at /pytorch/aten/src/ATen/native/Memory.cpp:46.)
  return data.pin_memory(device)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py:57: DeprecationWarning: The argument 'device' of Tensor.is_pinned() is deprecated. Please do not pass this argument. (Triggered internally at /pytorch/aten/src/ATen/native/Memory.cpp:31.)
  return data.pin_memory(device)
/usr/local/lib/python3.12/dist-packages/jupyter_client/sessi

Training Complete!


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Model saved to 'my_multilingual_model'


In [ ]:
from google.colab import drive
import shutil

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define path (It will create a folder 'IntelliDesk_Model' in your Drive)
drive_path = "/content/drive/MyDrive/IntelliDesk_Model"

# 3. Save the trained model there
print(f"💾 Saving model to {drive_path}...")
# Copy the local Colab folder to Google Drive
shutil.copytree("my_multilingual_model", drive_path, dirs_exist_ok=True)

print("✅ Model saved to Google Drive! You never have to train again.")

Mounted at /content/drive
💾 Saving model to /content/drive/MyDrive/IntelliDesk_Model...
✅ Model saved to Google Drive! You never have to train again.


In [ ]:
# Define complex test cases in English and Proper Hindi (Devanagari)
real_test_emails = [
    # 1. English: Critical Technical Issue
    """Subject: Critical Bug in Production - Checkout Failing
Hi Support, We are seeing a major issue on the checkout page.
When users click "Pay Now", the button spins endlessly and throws a "502 Bad Gateway" error.
Please investigate immediately.""",

    # 2. English: Billing Issue
    """Subject: Invoice #INV-9921 is incorrect
Hi Finance Team, I just received the invoice for February, but the amount seems wrong.
We were charged $500 for the Enterprise plan, but we downgraded to Pro last month.""",

    # 3. Hindi: Billing Issue (Proper Devanagari)
    """Subject: बिल में गलती है (Bill Error)
नमस्ते, मुझे लगता है कि मेरे पिछले महीने के बिल में गलती है।
मैंने अपना प्लान बदल दिया था लेकिन आपने मुझे ज्यादा पैसे चार्ज किए हैं। कृपया इसे चेक करें।""",

    # 4. Hindi: Technical Support (Proper Devanagari)
    """Subject: ऐप क्रैश हो रहा है (App Crashing)
नमस्ते टीम, जब भी मैं पीडीएफ (PDF) एक्सपोर्ट करने की कोशिश करता हूं, ऐप अचानक बंद हो जाता है।
स्क्रीन पर एरर 500 आ रहा है। कृपया मदद करें।""",

    # 5. Hindi: Access Request (Proper Devanagari)
    """Subject: एडमिन एक्सेस चाहिए (Need Admin Access)
राहुल के लिए एक नया अकाउंट बना दीजिये।
उसे डैशबोर्ड का फुल एक्सेस चाहिए क्योंकि वह कल से जॉइन कर रहा है।""",

    # 6. English: Feature Request
    """Subject: Request: Dark Mode
Hey team, our developers work late at night and the white background hurts our eyes.
Are there plans to add a Dark Mode soon?"""
]

# Run Inference
# The model automatically detects the language (English/Hindi) and classifies it.
preds = model(real_test_emails)
probs = model.predict_proba(real_test_emails)

print("\n" + "="*60)
print(f" TESTING {len(real_test_emails)} EMAILS (English + Proper Hindi) ")
print("="*60)

for i, email in enumerate(real_test_emails):
    # Get the email subject for cleaner display (first line only)
    subject = email.split("\n")[0]
    category = preds[i]
    confidence = probs[i].max().item()

    # Visual formatting for Pass/Fail confidence
    conf_str = f"{confidence:.2%}"

    print(f"📧 Email:      {subject[:50]}...")
    print(f"🤖 Prediction: [{category}]")
    print(f"📊 Confidence: {conf_str}")
    print("-" * 60)


 TESTING 6 EMAILS (English + Proper Hindi) 
📧 Email:      Subject: Critical Bug in Production - Checkout Fai...
🤖 Prediction: [Technical Support]
📊 Confidence: 67.37%
------------------------------------------------------------
📧 Email:      Subject: Invoice #INV-9921 is incorrect...
🤖 Prediction: [Billing/Invoice]
📊 Confidence: 98.57%
------------------------------------------------------------
📧 Email:      Subject: बिल में गलती है (Bill Error)...
🤖 Prediction: [Billing/Invoice]
📊 Confidence: 98.71%
------------------------------------------------------------
📧 Email:      Subject: ऐप क्रैश हो रहा है (App Crashing)...
🤖 Prediction: [Technical Support]
📊 Confidence: 99.37%
------------------------------------------------------------
📧 Email:      Subject: एडमिन एक्सेस चाहिए (Need Admin Access)...
🤖 Prediction: [Access Request]
📊 Confidence: 99.26%
------------------------------------------------------------
📧 Email:      Subject: Request: Dark Mode...
🤖 Prediction: [Feature Request]


/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn pydantic transformers torch

In [ ]:
from setfit import SetFitModel
from transformers import pipeline
from google.colab import drive
drive.mount('/content/drive')

print("⏳ Loading Classification Model (Multilingual)...")
# This loads your saved model. Make sure you ran Cell 4 at least once!
class_model = SetFitModel.from_pretrained("/content/drive/MyDrive/IntelliDesk_Model")

print("⏳ Loading Sentiment Model (RoBERTa)...")
# This downloads a pre-trained model expert in detecting emotions
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

print("✅ Models Loaded Successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ Loading Classification Model (Multilingual)...


The tokenizer you are loading from '/content/drive/MyDrive/IntelliDesk_Model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


⏳ Loading Sentiment Model (RoBERTa)...


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Models Loaded Successfully!


In [ ]:
def calculate_severity(subject, body, category):
    text = (subject + " " + body).lower()

    # Defaults
    priority = "P3"
    sla = "24 Hours"

    # --- Rule A: Category Baseline ---
    if category == "Feature Request":
        priority = "P4"
        sla = "72 Hours"
    elif category == "Hardware/Infrastructure":
        priority = "P2"
        sla = "4 Hours"

    # --- Rule B: Keywords (Expanded for your Requirements) ---
    p1_keywords = [
        "production down", "system failure", "data breach", "security alert",
        "emergency", "all users affected", "critical", "server down",
        "losing money", "business stopped"  # <--- Added Business Impact
    ]
    p2_keywords = [
        "broken", "bug", "deadline", "urgent", "blocking", "major issue",
        "cannot login", "app crash", "error 500",
        "immediately", "asap"               # <--- Added Time Sensitivity
    ]
    escalation_keywords = [
        "lawyer", "legal action", "sue", "cancel subscription",
        "refund immediately", "manager", "unacceptable"
    ]

    if any(word in text for word in p1_keywords):
        priority = "P1"
        sla = "1 Hour"
    elif any(word in text for word in p2_keywords) and priority != "P1":
        priority = "P2"
        sla = "4 Hours"

    # --- Rule C: Dynamic Signals (Tone & Punctuation) ---

    # 1. Yelling (Caps Lock > 40%)
    caps_ratio = sum(1 for c in body if c.isupper()) / len(body) if len(body) > 0 else 0
    is_yelling = caps_ratio > 0.4

    # 2. Exclamation Spam (!!!)
    is_urgent_punctuation = "!!!" in body or "?!?" in body

    # 3. AI Sentiment (Negative Vibe)
    sentiment = sentiment_analyzer(body[:512])[0]
    is_negative = sentiment['label'] == "negative" and sentiment['score'] > 0.85

    # Logic: If User is Angry, Yelling, or using "!!!", Bump Priority
    if (is_yelling or is_negative or is_urgent_punctuation) and priority == "P3":
        priority = "P2"
        sla = "4 Hours (Escalated due to Tone)"

    # --- Rule D: Auto-Escalation Logic ---

    # 1. Legal/Escalation Keywords -> Instant P1 or P2
    if any(word in text for word in escalation_keywords):
        priority = "P1"
        sla = "1 Hour (Escalated)"

    # 2. 3rd Follow-Up Check (Re: Re: Re:)
    # Counts if "re:" appears 3 or more times in the subject
    if subject.lower().count("re:") >= 3:
        priority = "P2"
        sla = "4 Hours (Repeated Follow-up)"

    return {
        "severity": priority,
        "sla": sla,
        "sentiment": "Negative" if is_negative else "Neutral/Positive",
        "flags": {
            "is_yelling": is_yelling,
            "has_urgent_punctuation": is_urgent_punctuation,
            "is_followup": subject.lower().count("re:") >= 3
        }
    }
print("✅ Logic Function Updated with All Rules!")

✅ Logic Function Updated with All Rules!


In [ ]:
# 1. Your Real Input Data
real_test_emails = [
    # 1. English: Critical Technical Issue
    """Subject: Critical Bug in Production - Checkout Failing
Hi Support, We are seeing a major issue on the checkout page.
When users click "Pay Now", the button spins endlessly and throws a "502 Bad Gateway" error.
Please investigate immediately.""",

    # 2. English: Billing Issue
    """Subject: Invoice #INV-9921 is incorrect
Hi Finance Team, I just received the invoice for February, but the amount seems wrong.
We were charged $500 for the Enterprise plan, but we downgraded to Pro last month.""",

    # 3. Hindi: Billing Issue (Proper Devanagari)
    """Subject: बिल में गलती है (Bill Error)
नमस्ते, मुझे लगता है कि मेरे पिछले महीने के बिल में गलती है।
मैंने अपना प्लान बदल दिया था लेकिन आपने मुझे ज्यादा पैसे चार्ज किए हैं। कृपया इसे चेक करें।""",

    # 4. Hindi: Technical Support (Proper Devanagari)
    """Subject: ऐप क्रैश हो रहा है (App Crashing)
नमस्ते टीम, जब भी मैं पीडीएफ (PDF) एक्सपोर्ट करने की कोशिश करता हूं, ऐप अचानक बंद हो जाता है।
स्क्रीन पर एरर 500 आ रहा है। कृपया मदद करें।""",

    # 5. Hindi: Access Request (Proper Devanagari)
    """Subject: एडमिन एक्सेस चाहिए (Need Admin Access)
राहुल के लिए एक नया अकाउंट बना दीजिये।
उसे डैशबोर्ड का फुल एक्सेस चाहिए क्योंकि वह कल से जॉइन कर रहा है।""",

    # 6. English: Feature Request
    """Subject: Request: Dark Mode
Hey team, our developers work late at night and the white background hurts our eyes.
Are there plans to add a Dark Mode soon?"""
]

print(f"{'PREDICTED CATEGORY':<25} | {'SEVERITY':<10} | {'SLA':<10} | {'SUBJECT'}")
print("-" * 100)

for email_text in real_test_emails:
    # -------------------------------------------------
    # STEP 1: PARSE SUBJECT & BODY
    # -------------------------------------------------
    # We split by the first newline character to separate Subject from Body
    parts = email_text.split("\n", 1)

    # Clean up the subject line (remove "Subject: " prefix if present)
    subject = parts[0].replace("Subject:", "").strip()
    body = parts[1].strip() if len(parts) > 1 else ""

    # -------------------------------------------------
    # STEP 2: PREDICT CATEGORY (AI Model)
    # -------------------------------------------------
    # Use the loaded 'class_model' from Cell 6B
    category_pred = class_model([email_text])[0]

    # -------------------------------------------------
    # STEP 3: CALCULATE SEVERITY (Logic Rules)
    # -------------------------------------------------
    # Pass the predicted category to your logic function
    urgency_data = calculate_severity(subject, body, category_pred)

    # -------------------------------------------------
    # STEP 4: PRINT RESULT
    # -------------------------------------------------
    print(f"{category_pred:<25} | {urgency_data['severity']:<10} | {urgency_data['sla'].split(' ')[0]:<10} | {subject[:40]}...")

PREDICTED CATEGORY        | SEVERITY   | SLA        | SUBJECT
----------------------------------------------------------------------------------------------------
Technical Support         | P1         | 1          | Critical Bug in Production - Checkout Fa...
Billing/Invoice           | P3         | 24         | Invoice #INV-9921 is incorrect...
Billing/Invoice           | P3         | 24         | बिल में गलती है (Bill Error)...
Technical Support         | P2         | 4          | ऐप क्रैश हो रहा है (App Crashing)...
Access Request            | P3         | 24         | एडमिन एक्सेस चाहिए (Need Admin Access)...
Feature Request           | P4         | 72         | Request: Dark Mode...


/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datet

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

# Initialize the App
app = FastAPI()

# Define the Input Format
class EmailRequest(BaseModel):
    subject: str
    body: str

@app.get("/")
def home():
    return {"status": "IntelliDesk AI is Running"}

@app.post("/process_email")
def process_email(req: EmailRequest):
    # -------------------------------------------------
    # STEP 1: PREDICT CATEGORY (AI Model 1)
    # -------------------------------------------------
    full_text = f"Subject: {req.subject}\n{req.body}"
    category_pred = class_model([full_text])[0]

    # Get confidence score (0.0 to 1.0)
    probs = class_model.predict_proba([full_text])
    confidence = float(probs.max().item())

    # -------------------------------------------------
    # STEP 2: CALCULATE SEVERITY (Logic Rules + Sentiment)
    # -------------------------------------------------
    # This calls the function you defined in Cell 6C
    severity_data = calculate_severity(req.subject, req.body, category_pred)

    # -------------------------------------------------
    # STEP 3: RETURN JSON RESPONSE
    # -------------------------------------------------
    return {
        "category": category_pred,
        "confidence": round(confidence, 4),

        # Data from your Logic Function
        "severity": severity_data['severity'],    # P1, P2, P3, P4
        "sla": severity_data['sla'],              # e.g., "4 Hours"
        "sentiment": severity_data['sentiment'],  # Negative vs Neutral

        # Diagnostic Flags (Helpful for Dashboard)
        "flags": severity_data['flags']           # {is_yelling, is_followup, etc.}
    }

print("✅ API App Defined Successfully!")

✅ API App Defined Successfully!


/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
import uvicorn
import threading
import nest_asyncio
from pyngrok import ngrok

# ---------------------------------------------------------
# 1. SETUP NGROK (PASTE TOKEN BELOW)
# ---------------------------------------------------------
# Get token from: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "your ngrok auth token"  # <--- PASTE HERE

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Kill any existing tunnels to prevent errors
ngrok.kill()

ngrok_tunnel = ngrok.connect(8000)

print(f"\n🚀 PUBLIC API URL: {ngrok_tunnel.public_url}")
print(f"👉 Send POST requests to: {ngrok_tunnel.public_url}/process_email")

def run_server():
    nest_asyncio.apply()
    uvicorn.run(app, port=8000)

# Start the server in a separate thread so Colab doesn't freeze
thread = threading.Thread(target=run_server)
thread.start()


🚀 PUBLIC API URL: https://hasteful-sophia-cardiologic.ngrok-free.dev
👉 Send POST requests to: https://hasteful-sophia-cardiologic.ngrok-free.dev/process_email
